# Выбор локации для скважины

Имеются пробы нефти из нескольких регионов, в каждом из которых по 10000 месторождений, где были измерены качество и объем нефти. Цель работы заключается в определнии лучшего региона для добычи нефти.

## 1. Загрузка библиотек

In [1]:
# Выгрузка стандартных билиотек
import warnings
from numpy.random import RandomState

# Выгрузка сторонних библиотек
import pandas as pd
#pd.get_option('mode.use_inf_as_null', True)
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns

#Выгрузка модулей библиотек для машинного обучения
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy import stats as st

# Объявление константы для "фиксации случайности"
rnd_st = 12345
state = RandomState(rnd_st)
# Настройка игнорирования ошибок
warnings.filterwarnings("ignore")

## 2. Загрузка и подготовка данных

### 2.1 Выгрузка данных

Выгрузим данные:

In [2]:
df_0 = pd.read_csv('geo_data_0.csv')

In [3]:
df_1 = pd.read_csv('geo_data_1.csv')

In [4]:
df_2 = pd.read_csv('geo_data_2.csv')

### 2.2 Первичный анализ данных

Просмотрим по 5 случайных строк из датасетов и общую информацию о столбцах

In [5]:
df_0.sample(5)

,id,f0,f1,f2,product
613,jipja,0.559992,0.594407,1.200322,103.068704
21669,fXqET,0.074760,-0.224921,-2.451357,82.519928
65370,ZGL93,0.893696,0.355648,3.282136,87.070657
77132,pEV40,1.010596,0.597912,3.658652,29.366977
6975,KTR6J,1.028349,0.412949,-1.435906,11.585772


In [6]:
df_0.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,100000,99990,fiKDv,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f0,100000.0,NaN,NaN,NaN,0.500419,0.871832,-1.408605,-0.07258,0.50236,1.073581,2.362331
f1,100000.0,NaN,NaN,NaN,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,NaN,NaN,NaN,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.00379
product,100000.0,NaN,NaN,NaN,92.5,44.288691,0.0,56.497507,91.849972,128.564089,185.364347


Видно, что в столбце 'id' 10 дублированных значений из 100000 строк.

In [7]:
df_1.sample(5)

,id,f0,f1,f2,product
15070,wVuEK,-14.868488,-7.781912,-0.002156,3.179103
97772,llthG,1.565177,-3.115006,0.005294,3.179103
96257,0FOfs,-4.086984,1.034609,4.997008,137.945408
4373,40wN4,-9.558297,-11.409497,0.998172,30.132364
95579,0YDId,-11.359189,-1.381277,2.991305,84.038886


In [8]:
df_1.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,100000,99996,wt4Uk,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f0,100000.0,NaN,NaN,NaN,1.141296,8.965932,-31.609576,-6.298551,1.153055,8.621015,29.421755
f1,100000.0,NaN,NaN,NaN,-4.796579,5.119872,-26.358598,-8.267985,-4.813172,-1.332816,18.734063
f2,100000.0,NaN,NaN,NaN,2.494541,1.703572,-0.018144,1.000021,2.011479,3.999904,5.019721
product,100000.0,NaN,NaN,NaN,68.825,45.944423,0.0,26.953261,57.085625,107.813044,137.945408


В столбце 'id' 4 дублированных значения.

In [9]:
df_2.sample(5)

,id,f0,f1,f2,product
51204,TVtzW,-0.689309,-3.487049,4.730334,189.459615
61752,8yZHa,0.120919,-1.937483,1.788844,66.800292
24246,GsJUd,0.621225,-1.109994,-0.118059,63.017184
23965,DKKb1,-1.709942,-2.502277,0.327630,63.739212
67309,LqisS,2.616963,-0.819205,5.289395,138.835583


In [10]:
df_2.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,100000,99996,VF7Jo,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f0,100000.0,NaN,NaN,NaN,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,NaN,NaN,NaN,-0.002081,1.730417,-7.08402,-1.17482,-0.009482,1.163678,7.844801
f2,100000.0,NaN,NaN,NaN,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,NaN,NaN,NaN,95.0,44.749921,0.0,59.450441,94.925613,130.595027,190.029838


Здесь ситуация аналогичная, что и в предыдущем датасете.

### 2.3 Предобработка данных

Удалим из датасетов столбец id: в будущем он не пригодится.

In [11]:
del_col = 'id'
df_0 = df_0.drop(del_col, axis=1)
df_1 = df_1.drop(del_col, axis=1)
df_2 = df_2.drop(del_col, axis=1)

Вывод: Была проведена предобработка, данные подготовлены к дальнейшему обучению.

## 3. Обучение и проверка модели

Напишем функцию, которая выделит из датафрейма признаки и целевые признаки, разделит их на тренировочные и валидационные выборки, масштабирует, проведет кросс-валидацию, проведет обучение линейной регрессии и выведет необходимые нам данные и значения:

In [12]:
def predict_regression(df):
    features = df.drop('product', axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                                  target,
                                                                                  test_size=.25, random_state=rnd_st)
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)

    model = LinearRegression()
    parameters = {'fit_intercept':[True,False],
                  'normalize':[True,False],
                  'copy_X':[True,False],
                  'n_jobs':[-1, 1, 2]}

    model = GridSearchCV(estimator=model,
                         param_grid=parameters,
                         scoring='neg_root_mean_squared_error',
                         cv=5)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    best_metric = mean_squared_error(target_valid, predicted_valid, squared=False)
    print(f'Истинный средний объем запасов скважины: {(target_valid.mean()) * 1000:14.3f} баррелей в год\n'
          f'Предсказанный средний объем запасов скважины: {(predicted_valid.mean()) * 1000:.3f} баррелей в год\n'
          f'Разница между ними составляет: {((predicted_valid.mean()) * 1000) - ((target_valid.mean()) * 1000):22.3f} баррелей в год\n'
          f'RMSE модели: {best_metric:39.3f}')

    return target_valid, predicted_valid, best_metric

In [13]:
target_valid_0, predicted_valid_0, rmse_0 = predict_regression(df_0)

Истинный средний объем запасов скважины:      92078.597 баррелей в год
Предсказанный средний объем запасов скважины: 92592.568 баррелей в год
Разница между ними составляет:                513.971 баррелей в год
RMSE модели:                                  37.579


In [14]:
target_valid_1, predicted_valid_1, rmse_1 = predict_regression(df_1)

Истинный средний объем запасов скважины:      68723.136 баррелей в год
Предсказанный средний объем запасов скважины: 68728.547 баррелей в год
Разница между ними составляет:                  5.411 баррелей в год
RMSE модели:                                   0.893


In [15]:
target_valid_2, predicted_valid_2, rmse_2 = predict_regression(df_2)

Истинный средний объем запасов скважины:      94884.233 баррелей в год
Предсказанный средний объем запасов скважины: 94965.046 баррелей в год
Разница между ними составляет:                 80.813 баррелей в год
RMSE модели:                                  40.030


Вывод: Самая низкая разница между истинным и предсказанным средним объемом запасов скважины и метрика RMSE у второго датафрейма, который отвечает за второй регион.
Самым Большим по среднему объему оказался третий регион, однако у него самое высокое значение RMSE

## 4. Подготовка к расчёту прибыли

### 4.1 Объявление констант

Опираясь на данные из условия задачи объявим константные переменные

In [16]:
DEVELOP_DOTS = 500 # При разведке региона исследуют 500 точек
ML_FEVELOP_DOTS = 200 # из которых с помощью машинного обучения выбирают 200 лучших для разработки.
BUDGET = 10 * 10**9 # Бюджет на разработку скважин в регионе
INCOME_BARREL = 450 # доход с барреля при нынешних ценах
INCOME = INCOME_BARREL * 1000 # Доход с каждой единицы продукта
PROBABILITY = 0.025 # Вероятность убытков

### 4.2 Минимальный объем производства для безубыточности

Посчитаем, сколько потребуется производить каждой точке сырья для безубыточной разработки

In [26]:
print(f'Для безубыточной разработки региона потребуется производить {BUDGET/(ML_FEVELOP_DOTS*INCOME):.2f} тысяч баррелей с каждой точки')

Для безубыточной разработки региона потребуется производить 111.11 тысяч баррелей с каждой точки


Значение минимального объема производства выше средних объемов трех рассмотренных регионов.

### 4.3 Максимальная прибыль с каждого региона

Напишем функцию, которая рассчитает максимальную прибыль с каждого региона. Возьмем в расчет 200 максимально продуктивных скважин.

In [18]:
def max_profit(target_valid, predicted_valid):
    predictions_max = pd.Series(predicted_valid).sort_values(ascending=False).head(200)
    target_max = target_valid.reset_index(drop=True)[predictions_max.index]
    income = target_max.sum() * INCOME
    return income

In [19]:
print(f'Первый регион: {(max_profit(target_valid_0, predicted_valid_0) - BUDGET) / 10**9:.2f} млрд. рублей')
print(f'Второй регион: {(max_profit(target_valid_1, predicted_valid_1) - BUDGET) / 10**9:.2f} млрд. рублей')
print(f'Третий регион: {(max_profit(target_valid_2, predicted_valid_2) - BUDGET) / 10**9:.2f} млрд. рублей')

Первый регион: 3.32 млрд. рублей
Второй регион: 2.42 млрд. рублей
Третий регион: 2.71 млрд. рублей


Вывод: По результатам расчета максимальной предполагаемой прибыли с каждого региона было установлено макисмальное значение в первом регионе.

По условию задачи известно, что на разработку будут рассматривать 500 скважин, а разрабатывать 200 лучших из них.

## Расчёт прибыли и рисков


Напишем функцию расчета прибыли. Похожа на функцию выше, но добавляется параметр count, отвечающий за количество активных строчек в валидационной выборке.

In [20]:
def revenue(target_valid, predicted_valid, count):
    predictions_max = pd.Series(predicted_valid).sort_values(ascending=False)
    target_max = target_valid.reset_index(drop=True)[predictions_max.index][:count]
    return target_max.sum() * INCOME

Напишем функцию, которая позволит отработать технику bootstrap. На выходе мы получим среднюю выручку, 2.5-й квартиль, доверительный интервал и вердикт, подходит нам расположение или нет.

In [22]:
def bootstrap_model(target, predictions, develop_dots):
    score = 0
    values = []
    for i in range(1000): # так как по условию bootstrap на 1000 выборок
        target_subsample = target.reset_index(drop=True).sample(n=develop_dots,
                                                                replace=True,
                                                                random_state=state
                                                                )
        predictions_subsample = predictions[target_subsample.index]
        inc = revenue(target_subsample, predictions_subsample, ML_FEVELOP_DOTS) - BUDGET
        values.append(inc)
        if inc < 0: # подсчитываем отрицательные прибыли
            score +=1

    values = pd.Series(values)

    mean = values.mean()
    lower = values.quantile(PROBABILITY)

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print(f"95%-ый доверительный интервал: {values.quantile(PROBABILITY)}, {values.quantile(1-PROBABILITY)}")
    pvalue = 1. * score / 1000
    if pvalue < PROBABILITY:
        print(f"Вероятность убытков равна {pvalue:.2%} и является меньше допустимой, регион подходит по критериям.")
    else:
        print(f"Вероятность убытков равна {pvalue:.2%} и является больше допустимой, регион не подходит по критериям.")

In [23]:
bootstrap_model(target_valid_0, predicted_valid_0, DEVELOP_DOTS)

Средняя выручка: 396164984.8023711
2.5%-квантиль: -111215545.89049526
95%-ый доверительный интервал: -111215545.89049526, 909766941.5534226
Вероятность убытков равна 6.90% и является больше допустимой, регион не подходит по критериям.


In [24]:
bootstrap_model(target_valid_1, predicted_valid_1, DEVELOP_DOTS)

Средняя выручка: 461155817.2772397
2.5%-квантиль: 78050810.7517417
95%-ый доверительный интервал: 78050810.7517417, 862952060.2637234
Вероятность убытков равна 0.70% и является меньше допустимой, регион подходит по критериям.


In [25]:
bootstrap_model(target_valid_2, predicted_valid_2, DEVELOP_DOTS)

Средняя выручка: 392950475.17060447
2.5%-квантиль: -112227625.37857565
95%-ый доверительный интервал: -112227625.37857565, 934562914.5511636
Вероятность убытков равна 6.50% и является больше допустимой, регион не подходит по критериям.


## Вывод

По результатам проведенной работы можно сделать вывод, что самый подходящий регион для разработки нефтяных меторождений это регион 2, так как только у него расчитанная вероятность убытков ниже допустимого значения.